In [46]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split


def load_data():
    #load the train files
    df = None
    
    y_train = []

    for i in range( 5,7 ):
        
        tmp = pd.read_csv( 'data/train%d.txt' % i, header=None, sep=" " )
                
        #build labels - one hot vector
        hot_vector = [ 1 if j == i else 0 for j in range(5,7) ]
        
        for j in range( tmp.shape[0] ):
            y_train.append( hot_vector )
        #concatenate dataframes by rows    
        if i == 0:
            df = tmp
        else:
            df = pd.concat( [df, tmp] )

    train_data = df.to_numpy()
    y_train = np.array( y_train )
    
    #load test files
    df = None
    
    y_test = []

    for i in range( 5,7 ):
        
        tmp = pd.read_csv( 'data/test%d.txt' % i, header=None, sep=" " )
        
        #build labels - one hot vector
        
        hot_vector = [ 1 if j == i else 0 for j in range(5,7) ]
        
        for j in range( tmp.shape[0] ):
            y_test.append( hot_vector )
        #concatenate dataframes by rows    
        if i == 0:
            df = tmp
        else:
            df = pd.concat( [df, tmp] )

    test_data = df.to_numpy()
    y_test = np.array( y_test )
    
    return train_data, test_data, y_train, y_test


#Load the dataset.
X_train, X_test, y_train, y_test = load_data()

#split our training dataset to train and validation set.
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, train_size=0.8, random_state=42) 


In [21]:
print(y_train)
print(y_test)

[[0 1]
 [1 0]
 [0 1]
 ...
 [1 0]
 [1 0]
 [0 1]]
[[1 0]
 [1 0]
 [1 0]
 ...
 [0 1]
 [0 1]
 [0 1]]


In [43]:
#print(x.values for x in X_train if x.values != 0)


<generator object <genexpr> at 0x0000021DEE515AC0>


In [47]:
#Preprocessing

#Reshaping our images from 28*28 to 784-element vectors.

def create_feature_vectors(num_of_features, data):
    
    vector = data.reshape([-1, num_of_features])
    return vector


#Creating training, test and validation vectors.
create_feature_vectors(784, X_train)
create_feature_vectors(784, X_test)
create_feature_vectors(784, X_val)    

#normalize data to get values between (0,1)
X_train = X_train.astype(float) / 255
X_test = X_test.astype(float) / 255
X_val = X_val.astype(float) / 255


In [40]:
print(X_train.shape)
print(X_val.shape)


(9071, 784)
(2268, 784)
784


In [17]:
#Meros B - Logistic Regression

#We add a column of 1s to each data vector.
#X_train = np.hstack( (np.ones((X_train.shape[0], 1) ), X_train))
#X_test = np.hstack( (np.ones((X_test.shape[0], 1) ), X_test))
#X_val = np.hstack( (np.ones((X_val.shape[0], 1) ), X_val))

m, n = X_train.shape
#We need this small value for the logs to avoid divided by zero warning.
epsilon = 1e-5

def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))

#We won't use L2 regularization on this version
def calculateCost(X, y, theta):
    
    h = sigmoid(X.dot(theta))
    cur_J = (y.T.dot(np.log(h + epsilon)) + (1-y).T.dot(np.log(1 - h + epsilon)))
    
    #calculate gradient
    gradient = X.T.dot(h-y)
    
    return cur_J, gradient

def LogisticRegression(X, y, X_val, y_val, epoch = 2000, alpha = 0.01):
    
    theta = np.zeros([X.shape[1], 2])
    print(theta.shape)
    m, n = X.shape
    
    J_train = []
    J_test = []
    
    for i in range(epoch):
        train_error, train_gradient = calculateCost(X, y, theta)
        test_error, _ = calculateCost(X_val, y_val, theta)
        
        #update theta parameters, using gradient ascent.
        theta += alpha * train_gradient
        
        J_train.append(train_error[0])
        J_test.append(test_error[0])
        
    return J_train, J_test, theta


def predict(theta, X):
    
    m = X.shape[0]
    
    #We will use 0.5 as the threshold to decide
    #on which class it belongs
    pred = np.zeros((m,1));
    pred = sigmoid(np.dot(X, theta))
    prob = pred
    pred = pred > 0.5 - 1e-6
    
    return pred, prob

    

In [78]:
#We train our model
J_train, J_test, theta = LogisticRegression(X_train, y_train, X_val, y_val)

#Predicting values
pred_train, prob_train = predict(theta, X_train)
pred_val, prob_val = predict(theta, X_val)

print( 'Accuracy of training set', np.mean( pred_train.astype('int') == y_train ) )
print( 'Accuracy of validation set', np.mean( pred_val.astype('int') == y_val ) )


(786, 2)


C:\Users\Nikos\AppData\Local\Temp\ipykernel_4984\1286219214.py:17: RuntimeWarning: divide by zero encountered in log
  cur_J = (y.T.dot(np.log(h)) + (1-y).T.dot(np.log(1 - h)))
C:\Users\Nikos\AppData\Local\Temp\ipykernel_4984\1286219214.py:11: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1.0 + np.exp(-z))


Accuracy of training set 0.4753610406790872
Accuracy of validation set 0.48897707231040566


In [43]:
#Here we will implement Logistic Regression
#but this time with L2 Regularization.

#the sigmoid function stays the same.

def calculateCost(X, y, theta, _lambda):
    
    h = sigmoid(X.dot(theta))
    
    #calculate cost function with regularization
    reg = (_lambda / 2.0) * np.sum(theta**2)
    cur_J = (y.T.dot(np.log(h + epsilon)) + (1-y).T.dot(np.log(1 - h + epsilon))) - reg
    
    #calculate gradient
    reg = _lambda * theta
    gradient = (X.T.dot(h-y) ) - reg
    
    return cur_J, gradient

def LogisticRegressionReg(X, y, X_val, y_val, _lambda = 0.0, epoch = 2000, alpha = 0.01):
    
    theta = np.zeros([X.shape[1], 2])
    print(theta.shape)
    #theta = np.zeros(X.shape[1]).reshape((-1,1))
    m, n = X.shape
    
    J_train = []
    J_test = []
    
    for i in range(epoch):
        train_error, train_gradient = calculateCost(X, y, theta, _lambda)
        test_error, _ = calculateCost(X_val, y_val, theta, _lambda)
        
        #update theta parameters, using gradient ascent.
        theta += alpha * train_gradient
        #print("theta is: ", theta)
        
        J_train.append(train_error[0])
        #print("J_train[i] is ", J_train[i])
        J_test.append(test_error[0])
        #print("J_test[i] is ", J_test[i])

        
    return J_train, J_test, theta




In [44]:
J_train, J_test, theta = LogisticRegressionReg(X_train, y_train, X_val, y_val)

#Predicting values
pred_train, prob_train = predict(theta, X_train)
pred_val, prob_val = predict(theta, X_val)

print( 'Accuracy of training set', np.mean( pred_train.astype('int') == y_train ) )
print( 'Accuracy of validation set', np.mean( pred_val.astype('int') == y_val ) )


(784, 2)


C:\Users\Nikos\AppData\Local\Temp\ipykernel_18848\73302964.py:12: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1.0 + np.exp(-z))


Accuracy of training set 0.4753610406790872
Accuracy of validation set 0.48897707231040566
